In [1]:
# EfficientFormer
import sys
sys.path.append("../")
import torch
from src.models.components.opp_finetune import EfficientFormerV2_finetuned

In [2]:
model = EfficientFormerV2_finetuned()

In [3]:
model.load_state_dict(torch.load("efficientformerV2_s1.pth"))

<All keys matched successfully>

In [4]:
model = model.eval()

In [5]:
torch.jit.script(model)

RuntimeError: 
Module 'Embedding' has no attribute 'new_proj' :
  File "/home/ds/Documents/Xomad/lightning-hydra-template/scripts/../src/models/components/efficientformerv2.py", line 333
    def forward(self, x):
        if self.light:
            out = self.new_proj(x) + self.skip(x)
                  ~~~~~~~~~~~~~ <--- HERE
        elif self.asub:
            out_conv = self.conv(x)


In [24]:
batch_size = 1
# Input to the model
x = torch.randn(batch_size, 3, 224, 224)
torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "onnx/efficientformerV2_s1.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=13,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}},
                  verbose=True)

In [25]:
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = 'onnx/efficientformerV2_s0.onnx'
model_quant = 'onnx/efficientformerV2_s0_dynamic_quantized.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant, op_types_to_quantize=['Gemm', 'MatMul'], extra_options={"MatMulConstBOnly":True})

Ignore MatMul due to non constant B: /[/base/network.4/network.4.7/token_mixer/MatMul]
Ignore MatMul due to non constant B: /[/base/network.4/network.4.7/token_mixer/MatMul_1]
Ignore MatMul due to non constant B: /[/base/network.4/network.4.8/token_mixer/MatMul]
Ignore MatMul due to non constant B: /[/base/network.4/network.4.8/token_mixer/MatMul_1]
Ignore MatMul due to non constant B: /[/base/network.5/attn/MatMul]
Ignore MatMul due to non constant B: /[/base/network.5/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/base/network.6/network.6.4/token_mixer/MatMul]
Ignore MatMul due to non constant B: /[/base/network.6/network.6.4/token_mixer/MatMul_1]
Ignore MatMul due to non constant B: /[/base/network.6/network.6.5/token_mixer/MatMul]
Ignore MatMul due to non constant B: /[/base/network.6/network.6.5/token_mixer/MatMul_1]
